# 환경 load & 라이브러리 임포트

In [14]:
import os
import langchain_chroma
import openai
from openai import OpenAI
import chromadb
import fitz  # PDF에서 텍스트 추출
from dotenv import load_dotenv
from fastapi import FastAPI, UploadFile, File
from sqlalchemy import create_engine, text
from sqlalchemy.ext.declarative import declarative_base
import pandas as pd
import psycopg2
import torch
from sklearn.metrics.pairwise import cosine_similarity
from langchain_text_splitters import RecursiveCharacterTextSplitter


# .env 파일 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
POST_DB_HOST = os.getenv("POST_DB_HOST")
POST_DB_NAME = os.getenv("POST_DB_NAME")
POST_DB_USER = os.getenv("POST_DB_USER")
POST_DB_PASSWD = os.getenv("POST_DB_PASSWD")
POST_DB_PORT = os.getenv("POST_DB_PORT")

In [3]:
# PostgreSQL 연결 엔진 생성
post_engine = create_engine(f'postgresql://{POST_DB_USER}:{POST_DB_PASSWD}@{POST_DB_HOST}:{POST_DB_PORT}/{POST_DB_NAME}')
db = psycopg2.connect(host=POST_DB_HOST, dbname=POST_DB_NAME,user=POST_DB_USER,password=POST_DB_PASSWD,port=POST_DB_PORT)
cursor = db.cursor()
db.autocommit = False

---

## data 불러오기

In [4]:
sql = """
        select rec_idx, company_nm, jd_text from saramin_recruit_detail
"""

cursor.execute(sql)

In [5]:
jd_text_list = cursor.fetchall()

In [6]:
db.commit()
cursor.close()
db.close()

In [7]:
# 텍스트 임베딩 함수
def get_embedding(text, model="text-embedding-3-large"):
    response = openai.embeddings.create(
        text = resume_text.replace("\n", "").strip(),
        model=model,
        input=text
    )
    return response.data[0].embedding

In [1]:
jd_text_list

NameError: name 'jd_text_list' is not defined

In [9]:
jd_text_list[0][2]

'\n채용공고 상세\n[코스닥 상장사] 경영기획본부 신입2007년 12월 11일에 설립된 그 외 기타 의료용 기기 제조업업종의 피부미용 의료기기 제조,중개,무역사업을 하는 코스닥,중소기업,주식회사,외부감사법인,병역특례 인증업체,수출입 기업 입니다.모집부문 및 상세내용모집부문상세내용공통 자격요건ㆍ학력 : 대졸 이상 (4년)ㆍ학점 : 3.8 이상경영기획본부전략기획팀 1명주요업무담당업무ㆍ지원직무주요 회의체 운영 및 경영진 지원 (회의 안건 준비, 회의록 작성 등)조직별 성과 데이터 집계 및 분석, 경영진 보고사업 전략 및 중장기 계획 수립 지원기타 경영지원 및 프로젝트성 업무 수행지원자격데이터 분석 및 보고서 작성 능력 (Excel, PowerPoint 활용)커뮤니케이션 및 조율 역량 보유컨설팅 및 전략 기획 관련 경험 우대\nㆍ기타 필수 사항\n우대사항✅ 전략기획, 경영관리 등 유관 업무 경험 보유자✅ 데이터 분석 및 리포팅 역량 우수자 (BI 툴, SQL 활용 가능자 우대)✅ 사업 전략 및 중장기 계획 수립 경험자✅ 주요 프로젝트 운영 경험 보유자근무조건ㆍ근무형태:정규직(수습기간)-3개월ㆍ근무일시:주 5일(월~금)ㆍ근무지역:(16827) 경기 용인시 수지구 신수로 767 분당수지 U-TOWER 19층(동천동) - 신분당선 동천 에서 200m 이내전형절차 서류전형 1차면접 2차면접(경우에 따라 생략 가능) 최종합격접수기간 및 방법ㆍ:2025년 3월 14일 (금) 14시~ 채용시ㆍ접수방법:사람인 입사지원ㆍ이력서양식:사람인 온라인 이력서ㆍ제출서류:유의사항ㆍ학력, 성별, 연령을 보지않는 블라인드 채용입니다. ㆍ입사지원 서류에 허위사실이 발견될 경우, 채용확정 이후라도 채용이 취소될 수 있습니다.ㆍ모집분야별로 마감일이 상이할 수 있으니 유의하시길 바랍니다.\n'

In [10]:
df_rec = pd.DataFrame(jd_text_list, columns=['rec_id', 'company_name', 'description'])

In [11]:
df_rec

,rec_id,company_name,description
0,50245530,(주)하이로닉,\n채용공고 상세\n[코스닥 상장사] 경영기획본부 신입2007년 12월 11일에 설...
1,50275785,세일즈웍스코리아(유),\n채용공고 상세\n
2,50275903,엔에이치엔(주),\n채용공고 상세\nNHN Dooray!올인원 협업 도구 두레이와 전자결재/게시판...
3,50254512,(주)글로벌스탠다드테크놀로지,\n채용공고 상세\n모집부문 및 자격요건\n 모집부문\n 경력사항\n 담당업무\n자...
4,49874934,(주)에이치비투자그룹,\n채용공고 상세\n
...,...,...,...
13848,50185544,(주)핀다,\n채용공고 상세\n 구분\n 상세내용\nBusiness\nPO\n(비교대출)\n(...
13849,50170082,오픈헬스케어(주),\n채용공고 상세\n\t\n\t 오픈헬스케어(주) ㅣ 전략기획본부 투자팀 - 경력\...
13850,50090229,(주)두나미스자산운용,\n채용공고 상세\n \n (주)두나미스자산운용\n \n펀드마케팅 \n경력3년이상(...
13851,50109575,(주)에코앤드림,\n채용공고 상세\n재무기획(FP&A) 담당자 채용\n# 모집부문 \n모집부문\n담...


---

# chroma db 적재 전 chunking

In [12]:
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [15]:
openai_client = OpenAI()
def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    return openai_client.embeddings.create(input=[text], model=model).data[0].embedding

In [16]:
# 토큰나이징
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # 약 1000 토큰(약 300~400 단어)
    chunk_overlap=200,    # 일부 문맥 보존
    separators=["\n\n", "\n", ".", " ", ""]
)

def split_texts(row):
    chunks = text_splitter.split_text(row["description"])
    return [(row["rec_id"], row["company_name"], chunk) for chunk in chunks]

---

# 다시 chromadb 유사도 검색 진행 코드

In [17]:
df_rec

,rec_id,company_name,description
0,50245530,(주)하이로닉,\n채용공고 상세\n[코스닥 상장사] 경영기획본부 신입2007년 12월 11일에 설...
1,50275785,세일즈웍스코리아(유),\n채용공고 상세\n
2,50275903,엔에이치엔(주),\n채용공고 상세\nNHN Dooray!올인원 협업 도구 두레이와 전자결재/게시판...
3,50254512,(주)글로벌스탠다드테크놀로지,\n채용공고 상세\n모집부문 및 자격요건\n 모집부문\n 경력사항\n 담당업무\n자...
4,49874934,(주)에이치비투자그룹,\n채용공고 상세\n
...,...,...,...
13848,50185544,(주)핀다,\n채용공고 상세\n 구분\n 상세내용\nBusiness\nPO\n(비교대출)\n(...
13849,50170082,오픈헬스케어(주),\n채용공고 상세\n\t\n\t 오픈헬스케어(주) ㅣ 전략기획본부 투자팀 - 경력\...
13850,50090229,(주)두나미스자산운용,\n채용공고 상세\n \n (주)두나미스자산운용\n \n펀드마케팅 \n경력3년이상(...
13851,50109575,(주)에코앤드림,\n채용공고 상세\n재무기획(FP&A) 담당자 채용\n# 모집부문 \n모집부문\n담...


In [18]:
def clean_description(text):
    if not isinstance(text, str):
        return ""
    return text.replace("\n", " ").strip()

df_rec["description"] = df_rec["description"].apply(clean_description)

In [19]:
df = df_rec[df_rec["description"].apply(lambda x: len(x) > 30)]

In [20]:
# rec_id 기준으로 description 통합
df = df.groupby("rec_id").agg({
    "company_name": "first",
    "description": lambda x: "\n".join(x)
}).reset_index()

In [ ]:
# ibal split 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""]
)

In [22]:
# chunk 생성
chunks = []
metas = []

for _, row in df.iterrows():
    rec_id, company_name, desc = row["rec_id"], row["company_name"], row["description"]
    split_texts = text_splitter.split_text(desc)
    for i, chunk in enumerate(split_texts):
        chunks.append(chunk)
        metas.append({
            "rec_id": str(rec_id),
            "company_name": company_name,
            "chunk_id": f"{rec_id}_{i}"
        })

In [23]:
def get_embedding(text, model="text-embedding-3-small"):
    return openai_client.embeddings.create(input=[text], model=model).data[0].embedding

In [24]:
from tqdm import tqdm
embeddings = [get_embedding(text) for text in tqdm(chunks)]

100%|██████████| 17641/17641 [2:16:36<00:00,  2.15it/s]  


In [26]:
# 백업용 
import json
import pickle

# 청크 저장장
with open("../data_backup/chunks.json", "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)

with open("../data_backup/metas.json", "w", encoding="utf-8") as f:
    json.dump(metas, f, ensure_ascii=False, indent=2)

# 피클 형식 (빠르고 이진 저장)
with open("../data_backup/embeddings.pkl", "wb") as f:
    pickle.dump(embeddings, f)

In [27]:
# 백업 파일 불러오기

# JSON 불러오기
with open("../data_backup/chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

with open("../data_backup/metas.json", "r", encoding="utf-8") as f:
    metas = json.load(f)

# 피클 불러오기
with open("../data_backup/embeddings.pkl", "rb") as f:
    embeddings = pickle.load(f)

KeyboardInterrupt: 

In [ ]:
embeddings

In [ ]:
# chromadb 클라이언트 설정
chroma_client = chromadb.HttpClient(host='', port=8000)  # 원하는 경로로 지정 가능
collection_name = "chroma_rec"

In [ ]:
# chroma db 상태 확인
chroma_client.heartbeat()
#chroma_client.reset()

In [ ]:
try:
    collection = chroma_client.get_collection(collection_name)
except:
    collection = chroma_client.create_collection(name=collection_name)

In [ ]:
# Chroma 적재
collection.add(
    documents=chunks,
    embeddings=embeddings,
    metadatas=metas,
    ids=[meta["chunk_id"] for meta in metas]
)
print(f"{len(chunks)}")


In [ ]:
len(embeddings)

In [ ]:
# chroma 내부에 데이터 확인
data = collection.get(include=["documents", "metadatas"], limit=5)

for doc, meta in zip(data["documents"], data["metadatas"]):
    print(f"[{meta['rec_id']} - {meta['company_name']}]\n{doc[:300]}...\n")

In [ ]:
print("총 문서 수:", collection.count())

In [ ]:
data = collection.get(include=["documents", "metadatas", "embeddings"], limit=1)

print(data["embeddings"][0][:10])  # 앞 10개만 보기

---

# Elastic Search

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

In [ ]:
es_client = Elasticsearch ("http://192.168.0.22:9200", basic_auth=("elastic", "ElastiC7276" ), timeout=30, max_retries=10, retry_on_timeout=True)

In [ ]:
es_client = Elasticsearch(
    "http://192.168.0.22:9200",
    basic_auth=("elastic", "ElastiC7276" ),
    request_timeout=60,
    max_retries=5,
    retry_on_timeout=True,
    connections_per_node=5    
)

In [ ]:
es_client.ping()

In [ ]:
index_name = "elastic_rec"

In [ ]:
# 인덱스 생성 (없을 때만)
if not es_client.indices.exists(index=index_name):
    es_client.indices.create(
        index=index_name,
        body={
            "mappings": {
                "properties": {
                    "description": {"type": "text"},         # 토큰화 검색
                    "rec_id": {"type": "keyword"},           # 정렬/필터링
                    "company_name": {"type": "keyword"}
                }
            }
        }
    )

In [ ]:
actions = [
    {
        "_index": index_name,
        "_id": meta["chunk_id"],
        "_source": {
            "rec_id": meta["rec_id"],
            "company_name": meta["company_name"],
            "description": chunk  # chunk 텍스트
        }
    }
    for chunk, meta in zip(chunks, metas)
]


In [ ]:
bulk(es_client, actions)

## 아래는 elastic search 연결 테스트

In [ ]:
# ping은 진짜 최소 응답이라 연결 가능 여부만 판단
es_client = Elasticsearch("http://192.168.0.22:9200", request_timeout=5)

try:
    print("ping:", es_client.ping())
except Exception as e:
    print("Error:", e)


In [ ]:
es_client.info()

In [ ]:
small_df = df.head(50)
index_name = "job_test_index"

In [ ]:
small_df

In [ ]:
if not es_client.indices.exists(index=index_name):
    es_client.indices.create(
        index=index_name,
        body={
            "mappings": {
                "properties": {
                    "description": {"type": "text"},
                    "rec_id": {"type": "keyword"},
                    "company_name": {"type": "keyword"}
                }
            }
        }
    )

In [ ]:
es_client.cluster.health()

In [ ]:
es_client.info()

---

# Hybrid Search 
1. 사용자 쿼리: 이력서/자소서 PDF → 텍스트 → 벡터
2. 검색:
    - ElasticSearch → 키워드 기반 검색 (precision↑)
    - ChromaDB      → 벡터 기반 유사도 검색 (recall↑)
3. 랭크퓨전 (rec_id 기준)
    - 3-1. rerank 할까 말까
4. Top-K 채용공고 추천

### 1. 사용자 쿼리 - 이력서

In [ ]:
path = "./resume_sample.pdf"
doc = fitz.open(path, filetype="pdf")
text = "\n".join([page.get_text("text") for page in doc])

In [ ]:
print(text)

### 여기서부터 

In [ ]:
# read pdf & pdf to text
def read_pdf(path):
    path = "./resume_sample.pdf"
    doc = fitz.open(path, filetype="pdf")
    resume_text = "\n".join([page.get_text("text") for page in doc])
    return resume_text

In [ ]:
def extract_text_from_pdf(path):
    doc = fitz.open(path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [ ]:
user_pdf_text = extract_text_from_pdf("resume_sample.pdf")
user_text_clean = user_pdf_text.replace("\n", " ").strip()

# 너무 길면 토큰 자르기
if len(user_text_clean) > 4000:
    user_text_clean = user_text_clean[:4000]  # OpenAI 임베딩 safe limit

query = user_text_clean  # 사용자 전체 문서 기반
query_embedding = get_embedding(query)

### 2. elastic search - 키워드 기반 검색

In [ ]:
def search_elastic(query, index_name, size=10):
    response = es_client.search(
        index=index_name,
        body={
            "query": {
                "multi_match": {
                    "query": query,
                    "fields": ["description", "company_name"]
                }
            }
        },
        size=size
    )
    return [
        {
            "rec_id": hit["_source"]["rec_id"],
            "score_elastic": hit["_score"]
        }
        for hit in response["hits"]["hits"]
    ]

### 3. chromadb 검색

### 4. rank fusion

In [ ]:
from collections import defaultdict

fusion_scores = defaultdict(lambda: {"score_elastic": 0, "score_chroma": 0})

for doc in search_elastic(user_query, index_name="job_test_index"):
    fusion_scores[doc["rec_id"]]["score_elastic"] = doc["score_elastic"]

for doc in res_chroma_docs:
    fusion_scores[doc["rec_id"]]["score_chroma"] = doc["score_chroma"]

# 가중 평균 or 단순 합산 (튜닝 가능)
fused = [
    {
        "rec_id": rec_id,
        "fusion_score": s["score_elastic"] * 0.5 + s["score_chroma"] * 0.5,
        "score_elastic": s["score_elastic"],
        "score_chroma": s["score_chroma"]
    }
    for rec_id, s in fusion_scores.items()
]

# Top-K 추천
top_k = sorted(fused, key=lambda x: x["fusion_score"], reverse=True)[:5]

### 5. 결과 확인

In [ ]:
# df는 rec_id 포함된 원본 데이터프레임
df_result = pd.DataFrame(top_k)
df_result = df_result.merge(df[["rec_id", "company_name", "description"]], on="rec_id", how="left")

df_result[["rec_id", "company_name", "fusion_score", "description"]].head()